## Extract the Oscar awards from the year 1980 until 2024.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
actor_types = [
    ('al', 'actor'),
    ('a_la', 'actriz'),
    ('al', 'actor_de_reparto'),
    ('a_la', 'actriz_de_reparto')
]

all_actors_movies = []

for art, actor_type in actor_types:
    url = f'https://es.wikipedia.org/wiki/Anexo:%C3%93scar_{art}_mejor_{actor_type}'

    # Realizar la solicitud GET a la página
    response = requests.get(url)

    # Crear el objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todas las tablas en la página
    tablas = soup.find_all('table')

    # Lista para almacenar los actores y películas de esta URL
    actors_movies = []

    # Iterar sobre cada tabla y extraer los nombres de los actores y películas
    for tabla in tablas:
        filas = tabla.find_all('tr')  # Encontrar todas las filas de la tabla
        for fila in filas:
            columnas = fila.find_all(['th', 'td'])  # Encontrar las columnas (celdas de encabezado y datos)
            datos_fila = [columna.text.strip() for columna in columnas]  # Extraer texto de cada celda
            # Añadir a la lista si hay datos en la fila y no es un encabezado
            if datos_fila and len(datos_fila) >= 4 and datos_fila[0].isdigit():
                actor_pelicula = {
                    'actor': datos_fila[2],
                    'pelicula': datos_fila[3]
                }
                actors_movies.append(actor_pelicula)

    # Añadir los actores y películas extraídos de esta URL a la lista principal
    all_actors_movies.append({
        'actor_type': actor_type,
        'actors_movies': actors_movies
    })

# Crear un DataFrame con los datos recolectados
data = []
for table_info in all_actors_movies:
    for actor_pelicula in table_info['actors_movies']:
        data.append([actor_pelicula['actor'], actor_pelicula['pelicula']])

oscars = pd.DataFrame(data, columns=['name', 'Película'])

oscars

,name,Película
0,Cillian Murphy,Oppenheimer
1,Brendan Fraser,The Whale
2,Will Smith,Rey Richard: Una familia ganadora
3,Anthony Hopkins,The Father
4,Joaquin Phoenix,Joker
...,...,...
442,Jane Darwell,The Grapes of Wrath
443,Hattie McDaniel,Lo que el viento se llevó
444,Fay Bainter,Jezabel
445,Alice Brady,In Old Chicago


In [4]:
import csv

oscars.to_csv('oscars.csv', index=False)

In [5]:
oscars = pd.read_csv('oscars.csv')

In [6]:
oscars = oscars.groupby("name").size().sort_values()
oscars = oscars.reset_index()
oscars.columns = ["name", "nominations"]
oscars

,name,nominations
0,Linda Hunt,1
1,Marie Dressler,1
2,Margaret Rutherford,1
3,Marcia Gay Harden,1
4,Lupita Nyong'o[1]​[2]​[3]​,1
...,...,...
311,Ingrid Bergman,3
312,Jack Nicholson,3
313,Walter Brennan,3
314,Katharine Hepburn,4


In [7]:
oscars.to_csv('oscars_df.csv')